## Contents

This notebook covers the basics of creating an event list object and carrying out various operations such as setting times, setting energies, and joining, storing and retrieving event lists.

## Setup

Import some useful libraries.

In [1]:
import numpy as np

Import some relevant stingray libraries.

In [2]:
from stingray.events import EventList
from stingray.lightcurve import Lightcurve

# Creating EventList object

### 1. From list of photon arrival times

Define photon arrival times.

In [3]:
times = [0.5, 1.5, 2.5, 3.5]

Create event list object by passing arrival times as argument.

In [4]:
ev = EventList(times)
ev.time

array([ 0.5,  1.5,  2.5,  3.5], dtype=float128)

### 2. From a Lightcurve

Create a lightcurve from arrays of time-stamps and counts.

In [12]:
times = np.arange(1000)
counts = np.random.rand(1000)*100
lc = Lightcurve(times, counts)

Now, create event list from the lightcurve created above.

In [13]:
ev = EventList(ncounts=1000)
ev.set_times(lc)
ev.time

array([-0.49530496, -0.4900938, -0.46263588, ...,  999.43383,  999.44848,
        999.45895], dtype=float128)

### 3. From a 'fits' file

In [14]:
ev = EventList()
ev = ev.read('events.fits', 'fits')
ev.time[1:10]

array([ 80000353.0,  80000056.0,  80000067.0,  80000637.0,  80000778.0,
        80000159.0,  80000631.0,  80000563.0,  80000817.0], dtype=float128)

# Binning to a Lightcurve

To create light curve out of event list, define interval, starting time and total duration of light curve.

In [15]:
ev = EventList(times)
lc = ev.to_lc(dt=1, tstart=0, tseg=4)
lc.counts

array([1, 1, 1, 2])

# Setting Energies

In order to set energies, a spectral distribution needs to be passed.

In [16]:
spectrum = [[1, 2, 3, 4, 5, 6],[1000, 2040, 1000, 3000, 4020, 2070]]

In [20]:
ev = EventList(ncounts=10)
ev.set_pha(spectrum)

# Joining EventLists

Two event lists can also be joined together.

In [21]:
ev1 = EventList(time=[1,2,3])

In [22]:
ev2 = EventList(time=[4,5])

In [24]:
ev = ev1.join(ev2)
ev.time

array([ 1.0,  2.0,  3.0,  4.0,  5.0], dtype=float128)

# IO

IO in all major formats including pickle, hdf5, fits and ascii is supported.

Below is an example of retrieving event list from a fits file.

In [26]:
ev = EventList()
ev = ev.read('events.fits', 'fits')
ev.time[1:10]

array([ 80000353.0,  80000056.0,  80000067.0,  80000637.0,  80000778.0,
        80000159.0,  80000631.0,  80000563.0,  80000817.0], dtype=float128)

Alternatively, we can used hdf5 to store and later retrieve data.

In [27]:
ev.write('ev.hdf5', 'hdf5')
ev = ev.read('ev.hdf5', 'hdf5')
ev.time[1:10]

array([ 80000353.0,  80000056.0,  80000067.0,  80000637.0,  80000778.0,
        80000159.0,  80000631.0,  80000563.0,  80000817.0], dtype=float128)